# ⭐ Tutorial: Hierarchical Risk Parity (HRP)

This notebook demonstrates the Hierarchical Risk Parity (HRP) algorithm, a modern portfolio allocation method from Marcos López de Prado (Chapter 16, 'Advances in Financial Machine Learning').

HRP is a powerful alternative to traditional Mean-Variance Optimization (MVO) and is robust to the instability of covariance matrices.

The algorithm works in three steps:
1.  **Tree Clustering:** Cluster assets based on their correlation, building a hierarchical tree (dendrogram).
2.  **Quasi-Diagonalization:** Reorder the covariance matrix so that similar assets are grouped together.
3.  **Recursive Bisection:** Iterate through the cluster hierarchy, splitting and distributing portfolio weights based on inverse cluster variance.

We will use the `RiskLabAI.optimization.hrp_alloc` function to perform this.

## 0. Setup and Imports

In [ ]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.cluster.hierarchy as sch
import warnings

# Third-party for data
!pip install yfinance
import yfinance as yf

# RiskLabAI Imports
from RiskLabAI.optimization.hrp import hrp_alloc
import RiskLabAI.utils.publication_plots as pub_plots

# Setup plotting and configuration
pub_plots.setup_publication_style()
warnings.filterwarnings('ignore')

## 1. Load Data

We'll download daily price data for a set of major ETFs from Yahoo Finance and calculate their log returns.

In [ ]:
tickers = ['SPY', 'TLT', 'GLD', 'QQQ', 'EEM', 'EFA', 'IWM', 'HYG']
start_date = '2010-01-01'
end_date = '2023-01-01'

data = yf.download(tickers, start=start_date, end=end_date)['Adj Close']
returns = np.log(data).diff().dropna()

print("Loaded returns data:")
returns.head()

## 2. Calculate Inputs

HRP requires two inputs:
1.  **Covariance Matrix:** Used for the final weight allocation.
2.  **Correlation Matrix:** Used for the tree clustering step.

In [ ]:
cov_matrix = returns.cov()
corr_matrix = returns.corr()

print("Correlation Matrix:")
display(corr_matrix.style.background_gradient(cmap='coolwarm'))

## 3. Visualize the Cluster Hierarchy

Before we run the allocation, let's visualize the first step. We can use `scipy.cluster.hierarchy` to compute and plot the dendrogram. This shows us how HRP 'sees' the asset universe.

In [ ]:
# 1. Calculate the linkage matrix
dist_matrix = np.sqrt((1 - corr_matrix.values) / 2.)
link = sch.linkage(dist_matrix, 'single')

# 2. Plot the dendrogram
fig, ax = plt.subplots(figsize=(10, 6))

dendrogram = sch.dendrogram(
    link, 
    labels=corr_matrix.columns,
    ax=ax,
    leaf_rotation=90
)

pub_plots.apply_plot_style(
    ax,
    title='Hierarchical Clustering of Assets (Dendrogram)',
    xlabel='Assets',
    ylabel='Distance'
)
ax.grid(axis='x')
plt.tight_layout()
plt.show()

## 4. Run HRP Allocation

Now we pass our covariance and correlation matrices to the `hrp_alloc` function to get the final portfolio weights.

In [ ]:
hrp_weights = hrp_alloc(cov_matrix, corr_matrix)

print("HRP Portfolio Weights:")
hrp_weights.sort_values(ascending=False).plot(kind='bar', figsize=(10, 6))
plt.title('HRP Portfolio Allocations')
plt.ylabel('Weight')
plt.xlabel('Asset')
plt.tight_layout()
plt.show()

**Conclusion:** The HRP algorithm successfully clustered the assets and produced a diversified set of portfolio weights, allocating capital across the different asset classes.